In [13]:
import time,re
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup as bs
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

def get_page_data(page_url):
    # page_url = "https://www.amazon.com/Best-Sellers-Toys-Games-Kids-Motor-Vehicle-Playsets/zgbs/toys-and-games/23627231011/ref=zg_bs_pg_1?_encoding=UTF8&pg=1"
    
    # starting the driver for parsing
    driver_path = "C:/Users/vkans/Documents/coursera-timeseries-deeplearning/chromedriver_win32.zip/chromedriver"
    service = Service(executable_path = driver_path)
    browser = webdriver.Chrome(service = service)

    chrome_options = Options()
    chrome_options.add_argument("--disable-extensions")
    chrome_options.add_argument("--headless") # chrome_options.headless = True (also works)

    # get the url
    browser.get(page_url)
    browser.implicitly_wait(0.5)

    # Scroll to the bottom, wait 1 seconds in between the scrolls, then continue scrolling.  It will continue scrolling until the page stops loading new data.
    SCROLL_PAUSE_TIME = 1
    lag_count = 0
    while True:
        lag_count+=1
        
        # Scroll down to bottom
        browser.execute_script("window.scrollBy(0, 300);")
        
        # Wait to load page and slow-down scrolling
        time.sleep(SCROLL_PAUSE_TIME)
        if lag_count>40: 
            break
    # the page is fully scrolled, then get the source code.
    source_data = browser.page_source
    browser.quit()

    # Throw your source data into BeautifulSoup and start parsing!
    bs_data = bs(source_data)
    return bs_data

def extract_data(html_page_data):
    # here, we parse the relevant data   
    product_list = []
    for item in html_page_data.select('.p13n-sc-uncoverable-faceout'):
        product_dict = {}
        product_dict['name'] = item.select('div > a')[1].get_text(strip=True)
        try:
            x = item.select('div > div')[2].get_text(strip=True)
            if x is not None:
                product_dict['price'] = (re.findall(r'\$[0-9]*[.][0-9]+',x)[0])
            else:
                product_dict['price'] = '$0.0'
        except IndexError:
            product_dict['price'] = '$0.0'   
        product_list.append(product_dict)
    df = pd.DataFrame(product_list)
    return df
        
def get_pages_url(html_page_data):
    base_url = 'https://www.amazon.com'
    pagination_section = html_page_data.find('ul',attrs = {"class" : "a-pagination"})
    pages_tag = pagination_section.find_all('a',href=True)
    page_urls = []
    for page in pages_tag:
        page_urls.append(base_url+page['href'])
    return list(set(page_urls))

In [35]:
df = pd.DataFrame()
lending_page = "https://www.amazon.com/Best-Sellers-Toys-Games-Kids-Motor-Vehicle-Playsets/zgbs/toys-and-games/23627231011/ref=zg_bs_pg_1?_encoding=UTF8&pg=1"
response = get_page_data(pg)

table_homedata = extract_data(response)
df = pd.concat([df,table_homedata],ignore_index=True)

page_urls = get_pages_url(response)
for pg_url in page_urls:
    if pg_url != lending_page:
        print(pg_url)
        page_data = get_page_data(pg_url)
        table_data = extract_data(page_data)
        df = pd.concat([df,table_data], ignore_index=True)    

https://www.amazon.com/Best-Sellers-Toys-Games-Kids-Motor-Vehicle-Playsets/zgbs/toys-and-games/23627231011/ref=zg_bs_pg_2?_encoding=UTF8&pg=2


In [36]:
df

,name,price
0,Hot Wheels Set Of 10 1:64 Scale Toy Trucks And...,$13.99
1,iHaHa 236 PCS Construction Toys Race Tracks fo...,$28.99
2,Fisher-Price Blaze and the Monster Machines Ra...,$17.99
3,TEMI Diecast Engineering Construction Vehicle ...,$25.99
4,"Paw Patrol, Chase’s 5-in-1 Ultimate Cruiser wi...",$78.49
...,...,...
95,"Race Car Track Toy for Kids, 200pcs Flexible T...",$39.99
96,Melissa & Doug Take-Along Town Play Mat (19.25...,$32.04
97,"Hot Wheels Monster Trucks, T-Rex Volcano Plays...",$98.00
98,Micro Machines Corvette Raceway Transforming C...,$40.07


In [37]:
df.to_excel('amazon-bestseller.xlsx', index= False)